In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import re

import geopandas as gpd
import requests,json


from shapely import geos
from shapely.geometry import Point
import fiona
from fiona.crs import from_epsg,from_string

from geopandas import GeoDataFrame
from shapely.geometry import Polygon
from geopandas import GeoSeries


from shapely import geometry
from shapely.geometry import shape,LineString,Point,Polygon

import osmnx as ox
import networkx as nx

import folium

from folium import IFrame

In [3]:
def rest(address):
    try:
        url = 'https://api.tomtom.com/search/2/geocode/{}.json?&key=h2n93GznJsnsDHtv2vPpdT9GmJijp9o9&countrySet=TWN&language=zh-TW&limit=1'.format(address)
        response = requests.get(url)
        data = response.text
        js = json.loads(str(data))
        return js['results'][0]['position']
    except Exception as e:
        print(f'An Error occurred: {e}')
        return {'lat':0,'lon':0}
    
def trans(gdf):
    gdf.crs = {'init' :'epsg:3824'} # 避免資料沒設，這邊再重新給一次
    gdf=gdf.to_crs({'init' :'epsg:4326'})
    return gdf

def utf2asc(s):
    return str(str(s).encode('ascii', 'xmlcharrefreplace'))[2:-1]

heading3 = """<h3>{}</h3>""".format

In [4]:
import warnings
warnings.filterwarnings("ignore")

# 設定row data的資料夾

In [94]:
import os
os.chdir(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data')

In [156]:
filename='HOLA2019'
save_file='HOLA'
save_year='2019'

# SQL有正確地址的資料

In [157]:
df_01=pd.read_csv(filename+"一般卡會員(有完整地址).csv",error_bad_lines=False,header=None)
df_01.columns=['id','addr','lon','lat']
df_01.shape

(800257, 4)

## 第一次抓取 村里

In [158]:
pattern = re.compile(r'(..[市,縣])(.{1,2}[市,區,鄉,鎮])(.{2,3}[里,村])')
location=df_01.addr.apply(lambda x:pattern.findall(x))

addr01=[]
addr02=[]
addr03=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0][0])
        addr02.append(i[0][1])
        addr03.append(i[0][2])        
    else:
        addr01.append(np.nan)
        addr02.append(np.nan)
        addr03.append(np.nan)  

In [159]:
df_01['縣市']=addr01
df_01['行政區']=addr02
df_01['村里']=addr03

### 第一次 有村里 資料

In [160]:
df_02=df_01.dropna()
df_02.shape

(800057, 7)

### 村里邊界資料

In [161]:
gdf=gpd.read_file(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\geo data\VILLAGE_MOI_1090423.shp',encoding='utf-8')
gdf=gdf.loc[:,['COUNTYNAME','TOWNNAME','VILLNAME','geometry']]
gdf=trans(gdf)
gdf=gdf.dropna().reset_index(drop=True)
print(gdf.shape)

gdf.columns=['縣市','行政區','村里','geometry']
gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))

df_02['縣市']=list(df_02['縣市'].apply(lambda x:x.replace('臺','台')))
df_02['行政區']=list(df_02['行政區'].apply(lambda x:x.replace('臺','台')))
df_02['村里']=list(df_02['村里'].apply(lambda x:x.replace('臺','台')))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舘','館'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('双','雙'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))
gdf.head()

(7760, 4)


,縣市,行政區,村里,geometry
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498..."
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492..."
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577..."
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557..."
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141..."


### 檢查1

In [162]:
site1=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].groupby(['村里'])['行政區'].nunique().reset_index(name='n').sort_values(by='n',ascending=False).reset_index(drop=True)
site1.head(5)

,村里,n
0,公?里,5
1,瓦?村,4
2,?林里,2
3,下?里,2
4,磚?里,2


In [163]:
number=0
for i in site1[site1.n>0]['村里']:
    print(number,i, df_02[df_02['村里']==i]['行政區'].unique())
    number+=1

0 公?里 ['板橋區' '北港鎮' '西區' '湖內區' '西螺鎮']
1 瓦?村 ['埔鹽鄉' '元長鄉' '麥寮鄉' '新園鄉']
2 ?林里 ['竹東鎮' '大園區']
3 下?里 ['彰化市' '東港鎮']
4 磚?里 ['西區' '彰化市']
5 ?子里 ['北屯區' '外埔區']
6 南?里 ['彰化市' '官田區']
7 ?榔里 ['清水區' '新屋區']
8 忠興里 ['將軍區' '湖內區']
9 珊湖里 ['頭份市']
10 玉?里 ['山上區']
11 爪峰里 ['瑞芳區']
12 爪?里 ['瑞芳區']
13 灰?里 ['中和區']
14 海風里 ['鳳山區' '清水區']
15 慈佑里 ['松山區']
16 瑞?村 ['梅山鄉']
17 永?里 ['三峽區']
18 水？村 ['北埔鄉']
19 水?村 ['北埔鄉']
20 新部里 ['永和區']
21 新?里 ['永和區']
22 文?村 ['竹崎鄉']
23 ??村 ['中埔鄉']
24 瓦?里 ['中和區']
25 瑪東里 ['七堵區' '安樂區']
26 萬昌里 ['中西區']
27 雙溪里 ['寶山鄉' '雙溪區']
28 里?里 ['關山鎮']
29 那拔里 ['新化區']
30 詔安里 ['和美鎮' '鹿港鎮' '白河區' '西螺鎮']
31 西和里 ['中西區' '將軍區']
32 蜣寮里 ['樹林區']
33 蔗?里 ['大肚區']
34 舊?村 ['埔心鄉']
35 崙?里 ['斗六市']
36 自治里 ['中壢區' '西區']
37 糖部里 ['萬華區']
38 糖?里 ['萬華區']
39 福松里 ['西區']
40 磚瑤里 ['彰化市']
41 石?村 ['中埔鄉']
42 白雪里 ['南區']
43 廈南村 ['萬丹鄉']
44 山?里 ['新化區']
45 崎?村 ['林邊鄉']
46 ?山里 ['阿蓮區']
47 ?田里 ['安南區']
48 ?溪村 ['梅山鄉']
49 ?洲里 ['永康區']
50 ?洞里 ['瑞芳區']
51 ?江里 ['麻豆區']
52 ?林村 ['滿州鄉']
53 ?廷里 ['土城區']
54 ?寮里 ['樹林區']
55 峰廷里 ['土城區']
56 ?子村 ['埔鹽鄉']
57 ?埔里 ['鳥松區']
58 ?南里 ['左營區'

In [164]:
site2=site1[site1.n>0]['村里'][30]
site3=df_02[df_02['村里']==site2]['行政區'].unique()

for i in site3:
    print(i,list(gdf[gdf['行政區']==i]['村里'][gdf[gdf['行政區']==i]['村里'].str.contains('山腳里')]))

和美鎮 []
鹿港鎮 []
白河區 []
西螺鎮 []


In [165]:
df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].groupby(['行政區','村里']).count().sort_values(by='id',ascending=False).head(15)

,,id,addr,lon,lat,縣市
行政區,村里,,,,,
新店區,五?里,736,736,736,736,736
北屯區,?子里,643,643,643,643,643
竹東鎮,上?里,501,501,501,501,501
永康區,?行里,425,425,425,425,425
三峽區,永?里,419,419,419,419,419
彰化市,南?里,394,394,394,394,394
土城區,?廷里,344,344,344,344,344
彰化市,下?里,265,265,265,265,265
中和區,瓦?里,243,243,243,243,243


In [166]:
gdf[gdf['村里']=='三廍村']

,縣市,行政區,村里,geometry
119,屏東縣,里港鄉,三廍村,"POLYGON ((120.52497 22.79960, 120.52385 22.799..."


In [167]:
df_02[df_02['村里']=='五?里']['行政區'].unique()

array(['新店區'], dtype=object)

### 字元調整

In [168]:
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖部里','糖廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖?里','糖廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚?里','磚磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚瑤里','磚磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下?里','下廍里'))

df_02.loc[df_02['行政區']=='彰化市',['村里']]=df_02.loc[df_02['行政區']=='彰化市',['村里']].apply(lambda x:x.replace('南?里','南瑶里'))
df_02.loc[df_02['行政區']=='官田區',['村里']]=df_02.loc[df_02['行政區']=='官田區',['村里']].apply(lambda x:x.replace('南?里','南廍里'))
#df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?里','南瑶里'))


df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙?里','崙峯里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?子里','廍子里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('石?村','石硦村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('上?里','上館里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('永?里','永館里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?村','瓦磘村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?里','瓦磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('五?里','五峯里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?行里','塩行里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?興里','塩興里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('公?里','公館里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?洲里','塩洲里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?龍里','雙龍里'))

#df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?林里','雞林里'))
df_02.loc[df_02['行政區']=='竹東鎮',['村里']]=df_02.loc[df_02['行政區']=='竹東鎮',['村里']].apply(lambda x:x.replace('?林里','雞林里'))

df_02.loc[df_02['行政區']=='大園區',['村里']]=df_02.loc[df_02['行政區']=='大園區',['村里']].apply(lambda x:x.replace('?林里','菓林里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?寮里','獇寮里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('新?里','新廍里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('蔗?里','蔗廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?埔里','坔埔里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?榔里','槺榔里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?江里','晋江里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?福村','雙福村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('爪?里','爪峯里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舊?村','舊館村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?村','南館村'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('灰?里','灰磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('三?村','三廍村'))


df_02.loc[df_02['行政區']=='將軍區',['村里']]=df_02.loc[df_02['行政區']=='將軍區',['村里']].apply(lambda x:x.replace('忠興里','忠嘉里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下部里','下廍里'))
df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']]=df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']].apply(lambda x:x.replace('?子村','廍子村'))
df_02.loc[df_02['行政區']=='四湖鄉',['村里']]=df_02.loc[df_02['行政區']=='四湖鄉',['村里']].apply(lambda x:x.replace('?子村','萡子村'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?田里','塩田里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('山?里','山脚里'))

### 第一次抓取 村里 與 geo data 合不合

In [169]:
#合
df_02_1=df_02[(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)

#不合
df_02_2=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_02_1.shape)
print(df_02_2.shape)

(799853, 7)
(204, 7)


### 不合的資料用經緯度 與村里邊界資料mapping

In [170]:
df_02_2.lon=df_02_2.lon.apply(lambda x:float(x))
df_02_2.lat=df_02_2.lat.apply(lambda x:float(x))

In [171]:
geom=[Point(data.lon,data.lat) for idx, data in df_02_2.iterrows()]
crs = {'init': 'epsg:4326'}
df_02_2 = gpd.GeoDataFrame(df_02_2, crs=crs, geometry=geom)
print(df_02_2.shape)

(204, 8)


In [172]:
df_02_3 = gpd.tools.sjoin(df_02_2,gdf, how="inner", op='intersects')
df_02_3.shape

(204, 12)

In [173]:
df_02_3=df_02_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
df_02_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
df_02_3.head()

,id,addr,lon,lat,縣市,行政區,村里
0,20181013000650,苗栗縣苑裡鎮山腳里17鄰錦山25號 ...,120.690862,24.418408,苗栗縣,苑裡鎮,山脚里
1,20051002000442,苗栗縣苑裡鎮山腳里4鄰山腳109號 ...,120.687331,24.413711,苗栗縣,苑裡鎮,山脚里
2,20180504000105,苗栗縣苑裡鎮山腳里5鄰山腳137號 ...,120.686868,24.414172,苗栗縣,苑裡鎮,山脚里
3,20030517002267,苗栗縣苑裡鎮山腳里10鄰山腳236號 ...,120.685573,24.415383,苗栗縣,苑裡鎮,山脚里
4,20010727000403,苗栗縣苑裡鎮山腳里20鄰錦山74之2號 ...,120.688963,24.419965,苗栗縣,苑裡鎮,山脚里


## 合併 合與不合的資料
## 獲得完整的第一次抓取 村里

In [174]:
df_02=pd.concat([df_02_1,df_02_3]).reset_index(drop=True)
df_02.shape

(800057, 7)

## 第一次抓取 無村里的資料

In [175]:
df_03=df_01[-df_01.id.isin(df_02.id)].dropna(subset=['lon'])
df_03.shape

(200, 7)

### 第二次抓取 村里

In [176]:
pattern = re.compile(r'(..[市,縣])(.{2,3}[市,區,鄉,鎮])(.{1,2}[里,村])')
location=df_03.addr.apply(lambda x:pattern.findall(x))

addr01=[]
addr02=[]
addr03=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0][0])
        addr02.append(i[0][1])
        addr03.append(i[0][2])        
    else:
        addr01.append(np.nan)
        addr02.append(np.nan)
        addr03.append(np.nan)  

In [177]:
df_03['縣市']=addr01
df_03['行政區']=addr02
df_03['村里']=addr03

### 第二次抓取 村里 與 geo data 合不合

In [178]:
#合
df_03_1=df_03[(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_03_1.shape)

#不合
df_03_2=df_03[-(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_03_2.shape)


(94, 7)
(106, 7)


### 不合的資料用經緯度 與村里邊界資料mapping

In [179]:
def to_float(x):
    try:
        return float(x)
    except:
        return 0

In [180]:
df_03_2.lon=df_03_2.lon.apply(to_float)
df_03_2.lat=df_03_2.lat.apply(to_float)

In [181]:
geom=[Point(data.lon,data.lat) for idx, data in df_03_2.iterrows()]
crs = {'init': 'epsg:4326'}
df_03_2 = gpd.GeoDataFrame(df_03_2, crs=crs, geometry=geom)
print(df_03_2.shape)

(106, 8)


In [182]:
df_03_3 = gpd.tools.sjoin(df_03_2,gdf, how="inner", op='intersects')
df_03_3.shape

(103, 12)

In [183]:
df_03_2[-df_03_2.id.isin(df_03_3.id)]

,id,addr,lon,lat,縣市,行政區,村里,geometry
20,20180204000965,埼玉?入間郡越生町如意 ...,139.313597,35.961175,NaN,NaN,NaN,POINT (139.31360 35.96117)
34,20180124000269,愛知?西尾市田貫町新明 ...,137.027095,34.872646,NaN,NaN,NaN,POINT (137.02709 34.87265)
59,20020410000073,A Choshimachi Kanazawa Ishikawa ...,136.702196,36.520963,NaN,NaN,NaN,POINT (136.70220 36.52096)


In [184]:
df_03_3=df_03_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
df_03_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
df_03_3.shape

(103, 7)

## 合併 合與不合的資料
## 獲得完整的第二次抓取 村里

In [185]:
df_03=pd.concat([df_03_1,df_03_3]).reset_index(drop=True)

## 合併 第一次抓取 有村里 與 無村里 資料，獲得SLQ有正確地址的資料

In [186]:
df_01=pd.concat([df_02,df_03]).reset_index(drop=True)
df_01.shape

(800254, 7)

# 存檔

In [187]:
df_01.to_csv('C:/Users/RT009223/TestriteGroup/滲透率專案/02_adjusted data/'+save_file+'會員位置(經整理_配對村里)_一般卡_'+save_year+'.txt',index=0)

# SLQ無正確地址的資料

In [188]:
df_04=pd.read_excel(filename+'一般卡會員(無完整地址).xlsx')
df_04.shape

(117599, 3)

In [189]:
df_04['addr']=df_04.addr.apply(lambda x:str(x))

# 區分有可用的地址 與 無可用的地址
## 有可用的地址

In [190]:
df_04_1=df_04[df_04.addr.apply(lambda x:len(x)>9)].reset_index(drop=True)
df_04_1.shape

(66729, 3)

In [191]:
pattern = re.compile(r'..[市,縣].+[路,街].+')
location=df_04_1.addr.apply(lambda x:pattern.findall((x)))

addr01=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0])      
    else:
        addr01.append(np.nan)

In [192]:
df_04_1.addr=addr01
df_04_1=df_04_1.dropna(subset=['addr']).reset_index(drop=True)
df_04_1.shape

(53315, 3)

In [193]:
df_04_1['lon']=float(0)
df_04_1['lat']=float(0)
df_04_1['check']=0
df_04_1.head()

,id,addr,zone,lon,lat,check
0,19991114000176,新北市永和區連城路469巷70弄9號,234.0,0.0,0.0,0
1,20010526000780,新北市新莊區後庄里育英街5巷7號復興路二段139號七樓之3,242.0,0.0,0.0,0
2,20090912000061,高雄市鳳山區830新康街166號11F,830.0,0.0,0.0,0
3,20100341347677,宜蘭縣宜蘭市頭城鎮頂埔路二段193巷12號,260.0,0.0,0.0,0
4,20040113000028,新北市新店區民安街３２８巷３號２樓,231.0,0.0,0.0,0


## 與現有 搜尋過 經緯度的資料 比對

In [194]:
df_get_back=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\get back.xlsx')
print(df_get_back.shape)
df_get_back.head()

(133014, 6)


,id,addr,zone,lon,lat,check
0,30012476701000,新北市貢寮區鳥崗街13號,228.0,122.00193,25.01326,1
1,20051030000358,新北市貢寮區福興村東興街４７號,228.0,121.99057,25.01205,1
2,20011022000749,新北市貢寮區春福連村臉香蘭街９號,228.0,121.98674,25.01985,1
3,20051219001868,新北市新店區東興街22號3樓DM退件,231.0,121.94981,25.01872,1
4,19980808000468,新北市貢寮區惠牴街25-2號,228.0,121.94576,25.01659,1


In [195]:
df_get_back=df_get_back=df_get_back.loc[:,['id','lon','lat','check']]

In [196]:
df_04_1=pd.merge(df_04_1.loc[:,['id','addr','zone']],df_get_back,on='id',how='left')

In [197]:
df_04_1.lon[df_04_1.lon.isna()]=0
df_04_1.lat[df_04_1.lat.isna()]=0
df_04_1.check[df_04_1.check.isna()]=0

In [198]:
df_04_1.head()

,id,addr,zone,lon,lat,check
0,19991114000176,新北市永和區連城路469巷70弄9號,234.0,0.0,0.0,0.0
1,20010526000780,新北市新莊區後庄里育英街5巷7號復興路二段139號七樓之3,242.0,0.0,0.0,0.0
2,20090912000061,高雄市鳳山區830新康街166號11F,830.0,0.0,0.0,0.0
3,20100341347677,宜蘭縣宜蘭市頭城鎮頂埔路二段193巷12號,260.0,0.0,0.0,0.0
4,20040113000028,新北市新店區民安街３２８巷３號２樓,231.0,0.0,0.0,0.0


## 與郵遞區號配對

In [199]:
df_zone=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\郵遞區代碼.xlsx')

In [200]:
df_04_1=pd.merge(df_04_1,df_zone,on='zone',how='left')

In [201]:
geom=[Point(data.lon,data.lat) for idx, data in df_04_1.iterrows()]
crs = {'init': 'epsg:4326'}
df_04_1 = gpd.GeoDataFrame(df_04_1, crs=crs, geometry=geom)
df_04_1.shape

(53315, 9)

In [202]:
df_04_1= gpd.tools.sjoin(df_04_1,gdf, how="left", op='intersects')
df_04_1.shape

(53315, 13)

In [203]:
df_04_1.head()

,id,addr,zone,lon,lat,check,縣市_left,行政區_left,geometry,index_right,縣市_right,行政區_right,村里
0,19991114000176,新北市永和區連城路469巷70弄9號,234.0,0.0,0.0,0.0,新北市,永和區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
1,20010526000780,新北市新莊區後庄里育英街5巷7號復興路二段139號七樓之3,242.0,0.0,0.0,0.0,新北市,新莊區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
2,20090912000061,高雄市鳳山區830新康街166號11F,830.0,0.0,0.0,0.0,高雄市,鳳山區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
3,20100341347677,宜蘭縣宜蘭市頭城鎮頂埔路二段193巷12號,260.0,0.0,0.0,0.0,宜蘭縣,宜蘭市,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
4,20040113000028,新北市新店區民安街３２８巷３號２樓,231.0,0.0,0.0,0.0,新北市,新店區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN


In [204]:
# 無 map 無 zone
df_04_1_1=df_04_1[(df_04_1['行政區_left'].isna()&df_04_1['村里'].isna())]
print(df_04_1_1.shape)

# 無 map 有 zone
df_04_1_2=df_04_1[(-(df_04_1['行政區_left'].isna())&(df_04_1['村里'].isna()))]
print(df_04_1_2.shape)

# 有 map 無有 zone
df_04_1_3=df_04_1[(df_04_1['行政區_left'].isna()&(-df_04_1['村里'].isna()))]
print(df_04_1_3.shape)

# 有 map 有 zone
df_04_1_4=df_04_1[(-(df_04_1['行政區_left'].isna())&(-df_04_1['村里'].isna()))]
print(df_04_1_4.shape)

(2553, 13)
(36591, 13)
(3594, 13)
(10577, 13)


###  有 map 有 zone 合不合

In [205]:
df_04_1_4_1=df_04_1_4[(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
print(df_04_1_4_1.shape)

df_04_1_4_2=df_04_1_4[-(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
print(df_04_1_4_2.shape)

(8719, 13)
(1858, 13)


### 調整各組資料

In [206]:
df_04_1_1=df_04_1_1.loc[:,['id']]

df_04_1_2=df_04_1_2.loc[:,['id','縣市_left','行政區_left']]
df_04_1_2.columns=['id','縣市','行政區']

df_04_1_3=df_04_1_3.loc[:,['id','縣市_right','行政區_right','村里']]
df_04_1_3.columns=['id','縣市','行政區','村里']

df_04_1_4_1=df_04_1_4_1.loc[:,['id','縣市_right','行政區_right','村里']]
df_04_1_4_1.columns=['id','縣市','行政區','村里']

df_04_1_4_2=df_04_1_4_2.loc[:,['id','縣市_left','行政區_left']]
df_04_1_4_2.columns=['id','縣市','行政區']

In [207]:
df_04_2=pd.concat([df_04_1_4_1,df_04_1_3,df_04_1_4_2,df_04_1_2,df_04_1_1])
df_04_2.shape

(53315, 4)

In [208]:
df_04_2.drop_duplicates(subset=['id']).shape

(53315, 4)

## 無可用地址

In [209]:
df_04_3=df_04[-df_04.id.isin(df_04_2.id)]
df_04_3.shape

(64284, 3)

In [210]:
df_04_3=pd.merge(df_04_3,df_zone,on='zone',how='left')

In [211]:
df_04_3_1=df_04_3[-df_04_3['行政區'].isna()]
print(df_04_3_1.shape)

df_04_3_2=df_04_3[df_04_3['行政區'].isna()]
print(df_04_3_2.shape)

(17922, 5)
(46362, 5)


In [212]:
df_04_3_1=df_04_3_1.loc[:,['id','縣市','行政區']]
df_04_3_2=df_04_3_2.loc[:,['id']]

# 完整 SLQ無正確地址的資料

In [213]:
df_04=pd.concat([df_04_2,df_04_3_1,df_04_3_2])
print(df_04.shape)
print(df_04.drop_duplicates(subset=['id']).shape)

(117599, 4)
(117599, 4)


# 存檔

In [214]:
df_04.to_csv('C:/Users/RT009223/TestriteGroup/滲透率專案/02_adjusted data/'+save_file+'會員位置(無正確地址的資料)_一般卡_'+save_year+'.txt',index=0)